In [1]:
import sys
sys.path.append('../src')

In [2]:
import numpy as np

In [3]:
x = np.array([1,1,1,2,2,2,5,25,1,1])

## Min

In [4]:
def calculate_min(signal):
    return np.min(signal)

In [5]:
print(calculate_min(x))

1


## Max

In [6]:
def calculate_max(signal):
    return np.max(signal)

In [7]:
print(calculate_max(x))

25


## Energy

$$
Energy = \sum_{x=x_0}^{x_0+N} y_x^2
$$


In [8]:
def calculate_energy(signal):
    return np.sum(np.power(signal,2))

In [9]:
print(calculate_energy(x))

667


## Line Length
$$
LineLength = \sum_{x=x_0+1}^{x_0+N}|y_x - y_{x-1}|
$$

In [10]:
def calculate_line_length(signal):
    offset_signal = signal[1:].copy()
    return(np.sum(np.abs(offset_signal-signal[:-1])))

In [11]:
print(calculate_line_length(x))

48


## Moving average

$$
MovingAverage = \frac{1}{N}\sum_{x=x_0}^{x_0+N}y_x
$$


In [12]:
def calculate_moving_avg(signal):
    return np.mean(signal)

In [13]:
print(calculate_moving_avg(x))

4.1


## Standard deviation
$$
StandardDeviation = \frac{1}{N}\sum_{x=x_0}^{x_0+N}(y_x-\mu)^2
$$


In [14]:
def calculate_std(signal):
    return np.std(signal)

In [15]:
print(calculate_std(x))

7.063285354564121


## Skewness

$$Skewness = \frac{\sqrt{N(N-1)}}{N(N-2)} g$$
$g = \frac{\sum_{x=x_0}^{x_0+N} (y_x - \mu)^3}{\sigma^3}$

In [16]:
def calculate_skewness(signal):
    std = np.std(signal)
    mean = np.mean(signal)
    n = len(signal)
    coeff = (n*(n-1))**0.5/(n*(n-2))
    skewness = np.sum(np.power(signal-mean,3))/std**3
    
    return coeff * skewness

In [17]:
print(calculate_skewness(x))

3.0129721592204315


## Kurtosis

$$Kurtosis = \frac{N-1}{(N-2)(N-3)}[(N+1) g + 6]$$

$g = \frac{\sum_{x=x_0}^{x_0+N} (y_x - \mu)^4}{\sigma^2} - 3$

In [18]:
def calculate_kurtosis(signal):
    std = np.std(signal)
    mean = np.mean(signal)
    n = len(signal)
    coeff = (n-1)/((n-2)*(n-3))
    kurtosis = np.sum(np.power(signal-mean,4))/std**2
    
    return coeff*((n+1)*kurtosis + 6)

In [19]:
print(calculate_kurtosis(x))

6780.540053474785


## Shannon entropy

$$ShannonEntropy = - \sum_{x=x_0}^{x_0+N} \text{freq}(y_x) \text{log}(\text{freq}(y_x))  $$

In [20]:
def calculate_shannon_entropy(signal, base=None):
    unique, counts = np.unique(signal, return_counts=True)
    n = len(signal)
    frequencies = counts/n
    return -np.sum(frequencies*np.log(frequencies))

In [21]:
print(calculate_shannon_entropy(x))

1.1682824501765625


## Local Binary Patterns

$$\sum_{r=0}^{p/2^{-1}}\left\{S[x[i+r-P/_{2}]-x[i]]2^{r}+S[x[i+r+1]-x[i]]2^{r+P/2}\right\} \tag{1}$$

Taken from: [Local binary patterns for 1-D signal processing](https://ieeexplore.ieee.org/document/7096717)

In [22]:
def calculate_lbp(signal):
    n = len(signal)
    middle_point = signal[n//2]
    
    signal = signal - middle_point
    
    # Thresholding
    signal[signal>=0] = 1
    signal[signal<0] = 0
    
    lbp = 0
    for i, e in enumerate(signal):
        if i != n//2:
            lbp += e*2**i
        
    return lbp

In [23]:
print(calculate_lbp(x))

216


### Phase synchrony

$$PLV={1\over N}\sqrt{\left(\sum_{i=0}^{N-1}sin(\Delta \phi_{i})\right)^{2}+\left(\sum_{i=0}^{N-1}cos(\Delta \phi_{i})\right)^{2}}$$

Taken from : [Phase-Synchronization Early Epileptic Seizure Detector](https://ieeexplore.ieee.org/document/6046232)

Neurons initiate electrical oscillations that are contained in multiple frequency bands such as alpha (8–12 Hz), beta (13–30 Hz) and gamma (40–80 Hz) and have been linked to a wide range of cognitive and perceptual processes. It has been shown that before and during a seizure the amount of synchrony between these oscillations from neurons located in different regions of the brain changes significantly. Thus, the amount of synchrony between multiple neural signals is a strong indicator in predicting or detecting seizures. To quantify the level of synchrony between two neural signals, a phase locking value (PLV) can be computed that accurately measures the phase-synchronization between two signal sites in the brain.

In [24]:
import scipy.signal as sig

def calculate_phase_synchrony(y1,y2):
    sig1_hill=sig.hilbert(y1)
    sig2_hill=sig.hilbert(y2)
    phase_y1=np.unwrap(np.angle(sig1_hill))
    phase_y2=np.unwrap(np.angle(sig2_hill))
    inst_phase_diff=phase_y1-phase_y2
    n = len(inst_phase_diff)
    
    sin_sum = np.sum(np.sin(inst_phase_diff))
    cos_sum = np.sum(np.cos(inst_phase_diff))
    
    return (1/n)*np.sqrt(np.power(sin_sum,2) + np.power(cos_sum,2))

In [25]:
inverted_x = x[::-1]
calculate_phase_synchrony(x,inverted_x)

0.13353001612654147